In [56]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [57]:
data = pd.read_csv('train.csv', encoding="utf-8")

In [58]:
# Normalize the pixel values
data = np.array(data)
data[:, 1:] = data[:, 1:] / 255.0

In [59]:
# Shuffle the data
np.random.shuffle(data)

In [60]:
# Split the data into training and development sets
data_dev = data[:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:]

data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:]

In [61]:
# Initializing parameters

def init_params():
    W1 = np.random.rand(10, 784)
    b1 = np.random.rand(10, 1)
    W2 = np.random.rand(10, 10)
    b2 = np.random.rand(10, 1)
    return W1, b1, W2, b2
    

In [62]:
# forward propagation

def ReLU(Z):
    return np.maximum(0, Z)

# def softmax(Z):
#     return np.exp(Z)/np.sum(np.exp(Z))

def stable_softmax(Z):
    Z -= np.max(Z)  # Subtract the maximum value for stability
    exp_Z = np.exp(Z)
    return exp_Z / np.sum(exp_Z)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = stable_softmax(Z2)
    return Z1, A1, Z2, A2

In [63]:
# backward propagation

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

In [64]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [65]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [66]:
def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if (i % 10 == 0):
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f'Iteration: {i}, Accuracy: {accuracy:.4f}')

    return W1, b1, W2, b2

# Increase the number of iterations and adjust the learning rate as needed
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 1000, 0.001)

Iteration: 0, Accuracy: 0.1051
Iteration: 10, Accuracy: 0.1050
Iteration: 20, Accuracy: 0.1049
Iteration: 30, Accuracy: 0.1049
Iteration: 40, Accuracy: 0.1050
Iteration: 50, Accuracy: 0.1051
Iteration: 60, Accuracy: 0.1051
Iteration: 70, Accuracy: 0.1051
Iteration: 80, Accuracy: 0.1041
Iteration: 90, Accuracy: 0.1039
Iteration: 100, Accuracy: 0.1033
Iteration: 110, Accuracy: 0.1032
Iteration: 120, Accuracy: 0.1023
Iteration: 130, Accuracy: 0.1023
Iteration: 140, Accuracy: 0.1021
Iteration: 150, Accuracy: 0.1020
Iteration: 160, Accuracy: 0.1018
Iteration: 170, Accuracy: 0.1017
Iteration: 180, Accuracy: 0.1017
Iteration: 190, Accuracy: 0.1023
Iteration: 200, Accuracy: 0.1033
Iteration: 210, Accuracy: 0.1033
Iteration: 220, Accuracy: 0.1032
Iteration: 230, Accuracy: 0.1033
Iteration: 240, Accuracy: 0.1035
Iteration: 250, Accuracy: 0.1034
Iteration: 260, Accuracy: 0.1029
Iteration: 270, Accuracy: 0.1014
Iteration: 280, Accuracy: 0.0885
Iteration: 290, Accuracy: 0.0873
Iteration: 300, Accur

KeyboardInterrupt: 